In [1]:
import sys
import torch
import pytorch_lightning as pl

import os
import warnings
import pathlib
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities.warnings import PossibleUserWarning

from midi.datasets import qm9_dataset, geom_dataset
from midi.diffusion_model import FullDenoisingDiffusion
from omegaconf import OmegaConf
from hydra import compose, initialize

In [2]:
config_path = "configs/"  
def load_hydra_configs(config_path):
    with initialize(config_path=config_path):
        cfg = compose(config_name="config")
    return cfg

In [3]:
cfg = load_hydra_configs(config_path)

C:\Users\xcdtl\AppData\Local\Temp\ipykernel_16228\1311385683.py:3: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=config_path):


In [4]:
# class cfg:
#     class dataset:
#         name = 'qm9'
#         datadir = 'train'
#         base_path = '/'
#         remove_h = True
#         pin_memory = True
#         random_subset = None
#         adaptive_loader = True
#     class train:
#         n_epochs = 1000
#         batch_size = 256
#         reference_batch_size = 300
#         lr = 0.0002
#         clip_grad = None          # float, None to disable
#         save_model = True
#         num_workers = 0
#         ema_decay = 0           # 'Amount of EMA decay, 0 means off. A reasonable value is 0.999.'
#         progress_bar = None
#         weight_decay = 1e-12
#         scheduler = None
#         seed = 0
#     class model:
#         # Model settings
#         type = 'discrete'
#         transition = 'marginal'                          # uniform or marginal
#         model = 'graph_tf'
#         diffusion_steps = 500
#         diffusion_noise_schedule = 'cosine'              # 'cosine', 'polynomial_2'
#         n_layers = 12
#         extra_features = None        # 'all', 'cycles', 'eigenvalues' or None
#         # Do not set hidden_mlp_E, dim_ffE too high, computing large tensors on the edges is costly
#         # At the moment (03/08), y contains quite little information
#         hidden_mlp_dims = {'X': 256, 'E': 64, 'y': 256, 'pos': 64}

#         # The dimensions should satisfy dx % n_head == 0
#         hidden_dims = {'dx': 256, 'de': 64, 'dy': 128, 'n_head': 8, 'dim_ffX': 256, 'dim_ffE': 64, 'dim_ffy': 256}

#         lambda_train = [3, 0.4, 1, 2, 0]
#         nu = {'p': 2.5, 'x': 1, 'c': 1, 'e': 1.5, 'y': 1}
#     class general:
#         # General settings
#         name = 'graph-tf-model'                   # Warning: 'debug' and 'test' are reserved name that have a special behavior
#         wandb = 'online'                # online | offline | disabled
#         gpus = 1                     # Multi-gpu is currently not implemented
#         resume = None            # If resume, path to ckpt file from outputs directory in main directory
#         test_only = None         # Use absolute path
#         check_val_every_n_epochs = 5
#         sample_every_val = 2
#         val_check_interval = None
#         samples_to_generate = 1024
#         samples_to_save = 20
#         chains_to_save= 1
#         log_every_steps= 50
#         number_chain_steps= 50        # Number of frames in each gif
#         faster_sampling= 1                              # At each sampling step, set s=t-faster sampling (1 for normal sampling)
#         final_model_samples_to_generate= 10000
#         final_model_samples_to_save= 10
#         final_model_chains_to_save: 5
#         cpus_per_gpu= 4
#         force_ray= False
#         evaluate_all_checkpoints= False
#         num_final_sampling= 5

        


In [5]:
dataset_config = cfg.dataset
data_module = qm9_dataset.QM9DataModule(cfg)
dataset_info = qm9_dataset.QM9infos(datamodule=data_module,cfg=cfg)

KeyboardInterrupt: 

In [ ]:
# gen=OmegaConf.load('configs/general/general_default.yaml')

# cfg.general = OmegaConf.merge(cfg,gen )

In [ ]:
model = FullDenoisingDiffusion(cfg=cfg, dataset_infos=dataset_info, train_smiles=[])

Marginal distribution of the classes: nodes: tensor([0.7231, 0.1151, 0.1591, 0.0026]) -- edges: tensor([0.7261, 0.2383, 0.0274, 0.0082, 0.0000]) -- charges: tensor([0.0078, 0.9705, 0.0218])


In [ ]:
params_to_ignore = ['module.model.train_smiles', 'module.model.dataset_infos']

In [ ]:
torch.nn.parallel.DistributedDataParallel._set_params_and_buffers_to_ignore_for_model(model, params_to_ignore)

In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath=f"checkpoints/{cfg.general.name}",
                                              filename='{epoch}',
                                              monitor='val/epoch_NLL',
                                              save_top_k=5,
                                              mode='min',
                                              every_n_epochs=1)

In [ ]:
callbacks=[]

In [ ]:
last_ckpt_save = ModelCheckpoint(dirpath=f"checkpoints/{cfg.general.name}", filename='last', every_n_epochs=1)
callbacks.append(checkpoint_callback)
callbacks.append(last_ckpt_save)

In [ ]:
torch.cuda.is_available()

False

In [ ]:
trainer = Trainer(gradient_clip_val=cfg.train.clip_grad, # Needed to load old checkpoints
                      accelerator='cpu',
                      devices=cfg.general.gpus,
                      max_epochs=cfg.train.n_epochs,
                      check_val_every_n_epoch=cfg.general.check_val_every_n_epochs,
                      fast_dev_run=cfg.general.name == 'debug',
                      enable_progress_bar=cfg.train.progress_bar,
                      callbacks=callbacks,
                      log_every_n_steps=50,
                      )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\xcdtl\anaconda3\envs\midi\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


In [ ]:
# FILEPATH: /f:/FYDP/EDM_Custom/colab_notebook.ipynb
trainer.fit(model, datamodule=data_module)



  | Name                  | Type                  | Params
----------------------------------------------------------------
0 | train_loss            | TrainLoss             | 0     
1 | train_metrics         | TrainMolecularMetrics | 0     
2 | val_metrics           | MetricCollection      | 0     
3 | val_nll               | NLL                   | 0     
4 | val_sampling_metrics  | SamplingMetrics       | 0     
5 | test_metrics          | MetricCollection      | 0     
6 | test_nll              | NLL                   | 0     
7 | test_sampling_metrics | SamplingMetrics       | 0     
8 | model                 | GraphTransformer      | 24.1 M
----------------------------------------------------------------
24.1 M    Trainable params
0         Non-trainable params
24.1 M    Total params
96.253    Total estimated model params size (MB)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: L

c:\Users\xcdtl\anaconda3\envs\midi\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: val/epoch_NLL: 503659200.00 -- val/pos_mse: 503658144.00 -- val/X_kl: 668.52 -- val/E_kl: 898.84 -- val/charges_kl: 461.67 
Val loss: 503659200.0000 	 Best val loss:  100000000.0000

Val epoch 0 ends
Starting epoch 0


c:\Users\xcdtl\anaconda3\envs\midi\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x000002020B7D3F70>> (for post_run_cell), with arguments args (<ExecutionResult object at 20249a26280, execution_count=17 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 20249a26be0, raw_cell="from omegaconf import OmegaConf

# FILEPATH: /f:/F.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/f%3A/FYDP/EDM_Custom/colab_notebook.ipynb#X21sZmlsZQ%3D%3D> result=None>,),kwargs {}:


c:\Users\xcdtl\anaconda3\envs\midi\lib\site-packages\pytorch_lightning\trainer\call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


TypeError: _pause_backend() takes 1 positional argument but 2 were given